In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
file = "data/daily_report/03-27-2020.csv"
df = pd.read_csv(file)
df.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active
0,South Carolina,US,3/27/20 22:14,34.223334,-82.461707,4,0,0,0
1,Louisiana,US,3/27/20 22:14,30.295065,-92.414197,8,1,0,0
2,Virginia,US,3/27/20 22:14,37.767072,-75.632346,2,0,0,0
3,Idaho,US,3/27/20 22:14,43.452658,-116.241552,54,0,0,0
4,Iowa,US,3/27/20 22:14,41.330756,-94.471059,1,0,0,0


In [4]:
us_df = df.where(df['Country_Region'] == 'US')

state_sum = us_df.groupby(['Province_State']).sum()
state_sum.head() 



,Lat,Long_,Confirmed,Deaths,Recovered,Active
Province_State,,,,,,
Alabama,2203.246784,-5809.578199,587.0,4.0,0.0,0.0
Alaska,1747.579877,-4229.319334,58.0,1.0,0.0,0.0
American Samoa,-14.271000,-170.132000,0.0,0.0,0.0,0.0
Arizona,505.138555,-1671.948482,665.0,13.0,0.0,0.0
Arkansas,2618.391704,-6932.548370,381.0,3.0,0.0,0.0


In [5]:
connection_string = "postgres:turambar1@localhost:5432/Covid-19"
engine = create_engine(f'postgresql://{connection_string}')

In [6]:
engine.table_names()

['coviddata', 'geo_states']

In [7]:

geo_states = pd.read_sql_table('geo_states', con=engine)
geo_states.head()

,name,abv,country,is_state,is_lower48,slug,latitude,longitude,population,area
0,Alabama,AL,US,y,y,alabama,32.806671,-86.791130,4779736,50744.00
1,Alaska,AK,US,y,n,alaska,61.370716,-152.404419,710231,571951.25
2,Arizona,AZ,US,y,y,arizona,33.729759,-111.431221,6392017,113634.57
3,Arkansas,AR,US,y,y,arkansas,34.969704,-92.373123,2915918,52068.17
4,California,CA,US,y,y,california,36.116203,-119.681564,37253956,155939.52


In [8]:
# geo_states = geo_states[['name', 'latitude', 'longitude']]

state_cords = state_sum.merge(geo_states, how='left', left_on='Province_State', right_on='name')

In [9]:
state_cords.head()

,Lat,Long_,Confirmed,Deaths,Recovered,Active,name,abv,country,is_state,is_lower48,slug,latitude,longitude,population,area
0,2203.246784,-5809.578199,587.0,4.0,0.0,0.0,Alabama,AL,US,y,y,alabama,32.806671,-86.791130,4779736.0,50744.00
1,1747.579877,-4229.319334,58.0,1.0,0.0,0.0,Alaska,AK,US,y,n,alaska,61.370716,-152.404419,710231.0,571951.25
2,-14.271000,-170.132000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,505.138555,-1671.948482,665.0,13.0,0.0,0.0,Arizona,AZ,US,y,y,arizona,33.729759,-111.431221,6392017.0,113634.57
4,2618.391704,-6932.548370,381.0,3.0,0.0,0.0,Arkansas,AR,US,y,y,arkansas,34.969704,-92.373123,2915918.0,52068.17


In [10]:
state_cords = state_cords[['name', 'latitude', 'longitude', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
state_cords.head()

,name,latitude,longitude,Confirmed,Deaths,Recovered,Active
0,Alabama,32.806671,-86.791130,587.0,4.0,0.0,0.0
1,Alaska,61.370716,-152.404419,58.0,1.0,0.0,0.0
2,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,Arizona,33.729759,-111.431221,665.0,13.0,0.0,0.0
4,Arkansas,34.969704,-92.373123,381.0,3.0,0.0,0.0


In [11]:
state_cords = state_cords.dropna()
state_cords.head()

,name,latitude,longitude,Confirmed,Deaths,Recovered,Active
0,Alabama,32.806671,-86.791130,587.0,4.0,0.0,0.0
1,Alaska,61.370716,-152.404419,58.0,1.0,0.0,0.0
3,Arizona,33.729759,-111.431221,665.0,13.0,0.0,0.0
4,Arkansas,34.969704,-92.373123,381.0,3.0,0.0,0.0
5,California,36.116203,-119.681564,4657.0,94.0,0.0,0.0


In [12]:
state_cords = state_cords.rename(columns={'name':'State', 'latitude':'Lat', 'longitude':'Long_'})
state_cords.head()

,State,Lat,Long_,Confirmed,Deaths,Recovered,Active
0,Alabama,32.806671,-86.791130,587.0,4.0,0.0,0.0
1,Alaska,61.370716,-152.404419,58.0,1.0,0.0,0.0
3,Arizona,33.729759,-111.431221,665.0,13.0,0.0,0.0
4,Arkansas,34.969704,-92.373123,381.0,3.0,0.0,0.0
5,California,36.116203,-119.681564,4657.0,94.0,0.0,0.0


In [13]:
state_cords.to_csv('data/daily_report/us_data_by_state.csv')

In [14]:
us_total = state_cords.sum()
us_total

State        AlabamaAlaskaArizonaArkansasCaliforniaColorado...
Lat                                                    2012.71
Long_                                                 -4760.31
Confirmed                                               101431
Deaths                                                    1574
Recovered                                                    0
Active                                                       0
dtype: object

In [15]:
us_total = pd.DataFrame(us_total)
us_total

,0
State,AlabamaAlaskaArizonaArkansasCaliforniaColorado...
Lat,2012.71
Long_,-4760.31
Confirmed,101431
Deaths,1574
Recovered,0
Active,0


In [16]:
us_total = us_total.transpose()


In [17]:
us_total

,State,Lat,Long_,Confirmed,Deaths,Recovered,Active
0,AlabamaAlaskaArizonaArkansasCaliforniaColorado...,2012.71,-4760.31,101431,1574,0,0


In [18]:

us_total = us_total.rename( columns={"State": "Country_Region"})
us_total

,Country_Region,Lat,Long_,Confirmed,Deaths,Recovered,Active
0,AlabamaAlaskaArizonaArkansasCaliforniaColorado...,2012.705961,-4760.308355,101431.0,1574.0,0.0,0.0


In [20]:
us_df = pd.DataFrame({
    "Country_Region": ["US"],
    "Lat": [39.099724],
    "Long_": [-94.578331],
    "Confirmed": [101431],
    "Deaths": [1574]
})
us_df

,Country_Region,Lat,Long_,Confirmed,Deaths
0,US,39.099724,-94.578331,101431,1574


In [22]:
us_df.to_csv('data/daily_report/us_total.csv')

In [24]:
df_noUS = df[df.Country_Region != "US"]
df_noUS.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active
3173,Alberta,Canada,3/27/20 23:27,53.9333,-116.5765,542,2,0,0
3175,Anhui,China,3/8/20 5:19,31.8257,117.2264,990,6,984,0
3176,Aruba,Netherlands,3/27/20 23:23,12.5211,-69.9683,33,0,1,32
3177,Australian Capital Territory,Australia,3/27/20 23:27,-35.4735,149.0124,62,0,1,61
3178,Beijing,China,3/27/20 5:30,40.1824,116.4142,569,8,408,153


In [25]:
df_noUS.to_csv('data/daily_report/data_minus_us.csv')